In [188]:
import math
import random
import numpy as np

In [189]:
# initialization
t = 0 # time step
m = 5 # number of nodes
r = 0.2 # poisson process rate lambda/m
rp = 0.2 # retransmission probability

In [190]:
def poisson_process(rate):
    p_0_arrive = math.exp(-1*rate)
    p_1_arrive = math.exp(-1*rate) * rate 
    
    result = -1
    
    p = random.random()
    if p < p_0_arrive:
        result = 0 # no packet was transmitted
    elif p < (p_0_arrive+p_1_arrive):
        result = 1 # one packet was transmitted
    else: 
        result = math.e # more than one packets were transmitted
        
    return result

In [191]:
def retransmission(retrans_rate):
    p = random.random()
    result = -1 # assume the node is still backlogged
    # if we are in the success region of retransmission rate 
    if p < retrans_rate: 
        result = 1 # then transmit
    return result

In [192]:
def update_status(node, channel_status):
    result = -1
    if channel_status == 0 or channel_status == 1: # if the channel is not in a collision
        if node == 0 or node == 1 : # the node transmitted 0 or 1 packet in the previous time unit
            # (and it was not pending for retransmission)
            # which means it is ready to transmit this time
            result = poisson_process(r) # determine how many packets to be transmitted this time
        else: # if the node is backlogged and wait for retransmission
            result = retransmission(rp)

    else: # if the channel is in a collision
        if node == 0: # if the node didn't try to send a packet
            result = 0 # no new arrivals
        else: # node is -1, 1, or e, the node had something to be sent
            result = retransmission(rp) # determine if it will be retransmitted
    return result
        

For the node status:
* 0 means no packet sending this time unit
* 1 means one packet sending this time unit
* e means multiple packets sending this time unit
* -1 means backlogged and wait for retransmission

In [193]:
random.seed(0)
stop = 10 # what time step to stop
nodes_status = [0]*m # -1 means pending for transmission
time_node_status = [[-1]*m]*stop # used to keep track of the nodes status each time
time_channel_status = [-1] * stop # used to keep track of the channel status each time
while t < stop:
    nodes_status = [update_status(node, channel_status) for node in nodes_status]
    print(nodes_status)
    
    channel_status = -1 # keep track of current channel
    if nodes_status.count(math.e) > 0 or nodes_status.count(1) > 1:
        # if a node already sent multiple packets or multiple nodes sent packets
        channel_status = math.e
    elif nodes_status.count(1) == 1:
        # only one node sent one packet
        channel_status = 1
    else: 
        # no node sent anything
        channel_status = 0
        
    time_node_status[t] = nodes_status # keep track of node status
    time_channel_status[t] = channel_status # keep track of channel status
    t += 1 # increment time step

[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 1, 2.718281828459045, 0, 1]
[0, -1, -1, 0, -1]
[0, -1, 1, 0, -1]
[1, -1, 0, 1, -1]
[-1, -1, 0, 1, -1]
[-1, -1, 0, 0, -1]


In [195]:
time_node_status
time_channel_status

[0, 1, 0, 1, 2.718281828459045, 0, 1, 2.718281828459045, 1, 0]